<a href="https://colab.research.google.com/github/tomonari-masada/course2023-nlp/blob/main/13_finetuning_GPTQ_LLM_with_LoRA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# GPTQで量子化されたLLMをLoRAでfinetuningする

## LLMを効率よくカスタマイズするには？
* パラメータ数が数B（数十億）のLLMをfinetuningするのは、大変。
* しかし、あらかじめ量子化されたモデルを、LoRAでfinetuningするなら、まあ、手に負える。
* ただし・・・
  * そもそも、量子化されたモデルは、元のモデルほどは性能が良くない。
* さらに・・・
  * LoRAでfinetuningするよりも、元のモデルを直接finetuningするほうが、性能は上がる。

## 参考資料
* 今回の授業資料の元になっているmediumの記事
  * https://dsmonk.medium.com/training-and-deploying-of-quantized-llms-with-lora-and-gptq-part-2-2-ec7b54659c9e

* LLMのfinetuningに関するその他の参考資料
  * https://medium.com/@pazuzzu/in-depth-llm-fine-tuning-guide-efficiently-fine-tune-and-use-zephyr-7b-beta-assistant-using-lora-e23d8151e067
  * https://huggingface.co/docs/trl/main/en/sft_trainer
  * https://blog.gopenai.com/fine-tuning-mistral-7b-instruct-model-in-colab-a-beginners-guide-0f7bebccf11c
* SFTTrainerによるfinetuningのコードの雛形
  * https://github.com/huggingface/trl/blob/main/examples/scripts/sft.py
* LLMを効率的に動かすこと全般に関するHugging Faceのチュートリアル（finetuningは関係ない）
  * https://huggingface.co/docs/transformers/llm_tutorial_optimization

## インストール

* transformersライブラリは最新のものに更新した方がいいかも。
  * finetuningでlossがゼロになったら、原因はおそらくこれ。

In [ ]:
!pip install -U git+https://github.com/huggingface/transformers trl accelerate torch bitsandbytes peft datasets auto-gptq optimum

**ここでセッションを再起動する。**

## 再現性の確保
* transformersのset_seed関数
  * randomもnumpyもPyTorchも、ちゃんと乱数のシードを設定しているようです。
  * https://github.com/huggingface/transformers/blob/main/src/transformers/trainer_utils.py#L85

In [ ]:
from transformers import set_seed

set_seed(123)

## データセット
* finetuningにはライブドアニュースコーパスの`title`フィールドを使う。
  * `content`フィールドを使ってみてもいいです。

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42, # 再現性の確保
    shuffle=True,
    trust_remote_code=True,
)

In [ ]:
dataset

In [ ]:
dataset["train"]["title"][0]

## モデル
* 前回と同様、`weblab-10b-instruction-sft`をGPTQで量子化したモデルを使う。
  * https://huggingface.co/dahara1/weblab-10b-instruction-sft-GPTQ

In [ ]:
model_id = "dahara1/weblab-10b-instruction-sft-GPTQ"
model_basename = "gptq_model-4bit-128g"

* 今回主に参考にしたのは、下の記事。
  * https://dsmonk.medium.com/training-and-deploying-of-quantized-llms-with-lora-and-gptq-part-2-2-ec7b54659c9e

* gradient checkpointingについては、下記を参照。
  * https://huggingface.co/docs/transformers/main/en/perf_train_gpu_one#gradient-checkpointing

In [ ]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
from peft import prepare_model_for_kbit_training

model = AutoGPTQForCausalLM.from_quantized(
    model_id,
    model_basename=model_basename,
    device="cuda:0",
    )

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Gradient Checkpointingを行うために必要な設定
model.config.use_cache = False

* モデルのどの部分をfinetuningするか？
  * 今回は、attention部分で使われている行列をfinetuningする。

In [ ]:
model

## LoRAの設定

* LoRAはPEFTのなかで最もよく使われている手法。
  * トランスフォーマモデルを構成する様々なパラメータ行列をfinetuningする。
  * ただし、元のモデルはfreezeさせて、finetuningによる差分だけ学習する。
  * そして、差分そのものを学習するのではなく、差分の低ランク近似を学習する。
* 詳しくは、原論文を参照。
  * https://arxiv.org/abs/2106.09685

* `lora_alpha`をLoRAのランクの2倍にするというのは、rule of thumb。
  * https://lightning.ai/pages/community/lora-insights/

In [ ]:
from peft import LoraConfig, get_peft_model

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig(
    r=16,
    lora_alpha=32,
    target_modules=["attention.query_key_value", "attention.dense"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, config)

In [ ]:
def print_trainable_parameters(model):
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )
print_trainable_parameters(model)

## finetuningの設定

* `remove_unused_columns=False`
  * これを追加した上で・・・
  * finetuningの実行前に、余分なcolumnsを手動で削除する（後出）。
  * こうしないと、なぜかデータセットに関するエラーが出てしまう。


In [ ]:
from transformers import TrainingArguments

args = TrainingArguments(
    per_device_train_batch_size=4,
    gradient_accumulation_steps=4,
    warmup_steps=10,
    max_steps=100,
    learning_rate=1e-4,
    fp16=True, #use mixed precision training
    logging_steps=1,
    output_dir="outputs_gptq_training",
    optim="adamw_hf",
    save_strategy="epoch",
    remove_unused_columns=False,
    # 以下の2行がvalidation setによる評価のための設定
    evaluation_strategy="steps",
    eval_steps=10,
    )

* 上の設定は適当に決めたもの。
* これをチューニングしてはじめて性能が出る。
* 以下のハイパーパラメータは特に重要。
  * `learning_rate`
  * `per_device_train_batch_size`
  * `gradient_accumulation_steps`

## `SFTTrainer`

* TRL = Transformer Reinforcement Learning
  * https://huggingface.co/docs/trl/
  * TRLは強化学習によってLLMをfinetuningするためのライブラリだが・・・
  * 今回は、単にSFT (supervised finetuning)をおこなうために使うだけ。

In [ ]:
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    args=args,
    train_dataset=dataset["train"],
    eval_dataset=dataset["validation"],
    peft_config=config,
    dataset_text_field="title",
    tokenizer=tokenizer,
    packing=False,
    max_seq_length=256,
    )

## 学習の実行

* なぜかこの操作が必要。

In [ ]:
trainer.train_dataset = trainer.train_dataset.remove_columns(['url', 'date', 'content', 'category', 'title'])
trainer.eval_dataset = trainer.eval_dataset.remove_columns(['url', 'date', 'content', 'category', 'title'])

* 今回の設定でのfinetuningには7分弱かかる。
* finetuning終了後に表示される情報の意味は、下記を参照。
  * https://huggingface.co/docs/transformers/main_classes/callback#transformers.TrainerState

In [ ]:
trainer.train()

In [ ]:
trainer.model.save_pretrained(".")

**評価の前にセッションを再起動する**

# 評価

## データセット

In [ ]:
from datasets import load_dataset

dataset = load_dataset(
    "shunk031/livedoor-news-corpus",
    train_ratio=0.8,
    val_ratio=0.1,
    test_ratio=0.1,
    random_state=42, # 再現性の確保
    shuffle=True,
    trust_remote_code=True,
)

In [ ]:
dataset["train"]["title"][0]

## 保存したLoRAモデルの読み込み

In [ ]:
model_id = "dahara1/weblab-10b-instruction-sft-GPTQ"
model_basename = "gptq_model-4bit-128g"

* まずモデル本体をロードする。

In [ ]:
from transformers import AutoTokenizer
from auto_gptq import AutoGPTQForCausalLM
from peft import prepare_model_for_kbit_training

model = AutoGPTQForCausalLM.from_quantized(
    model_id,
    model_basename=model_basename,
    device="cuda:0",
    )

tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token

# Gradient Checkpointingを行うために必要な設定
model.config.use_cache = False

* 次にLoRAをロードする。

In [ ]:
from peft import LoraConfig, get_peft_model, PeftConfig

model.gradient_checkpointing_enable()
model = prepare_model_for_kbit_training(model)

config = LoraConfig.from_pretrained(".")

model = get_peft_model(model, config)

## コーパスの埋め込み

* average poolingをおこなう関数

In [ ]:
def average_pool(last_hidden_states, attention_mask):
  last_hidden = last_hidden_states.masked_fill(~attention_mask[..., None].bool(), 0.0)
  return last_hidden.sum(dim=1) / attention_mask.sum(dim=1)[..., None]

* 全てのテキストの埋め込み
  * 手元のRTX3090で実行すると、1分半で終わる。

In [ ]:
from tqdm.notebook import tqdm
import torch

batch_size = 16

embeddings_list = {}
for key in dataset.keys():
  corpus = dataset[key]["title"]
  offset = 0
  embeddings_list[key] = list()
  for offset in tqdm(range(0, len(corpus), batch_size)):
    batch_dict = tokenizer(
        corpus[offset:offset+batch_size],
        padding=True, truncation=True, return_tensors='pt'
        ).to("cuda")
    with torch.no_grad():
      last_hidden_state = model(
          input_ids=batch_dict.input_ids,
          attention_mask=batch_dict.attention_mask,
          output_hidden_states=True,
          ).hidden_states[-1].cpu()
    tmp_embeddings = average_pool(last_hidden_state, batch_dict.attention_mask.cpu())
    embeddings_list[key].append(tmp_embeddings)
    offset += batch_size

In [ ]:
embeddings = {}
for key in dataset.keys():
  embeddings[key] = torch.concat(embeddings_list[key]).type(torch.float32)
  print(key, embeddings[key].shape)

In [ ]:
for key in dataset.keys():
  torch.save(embeddings[key], f"livedoor_weblab-10b-instruction-sft-GPTQ_finetuned_{key}.pt")

## 1-NNによる分類

In [ ]:
import torch.nn.functional as F

similarities = torch.matmul(
    F.normalize(embeddings["validation"], dim=-1),
    F.normalize(embeddings["train"], dim=-1).t()
)

In [ ]:
sorted_indices = torch.argsort(similarities, descending=True).cpu()

In [ ]:
category = {}
for key in dataset.keys():
  category[key] = torch.tensor(dataset[key]["category"])

In [ ]:
print(category["validation"][0], category["train"][sorted_indices[0,:20]])

In [ ]:
(category["train"][sorted_indices[:,0]] == category["validation"]).sum() / len(category["validation"])

* finetuningでvalidation lossをチェックしつつ、ハイパーパラメータをチューニングするのが正しい手順。